# Нумерация каждого сустава

<img src="https://i.imgur.com/3j8BPdc.png" style="height: 300px">

# Код для определения человека в определенном месте в кадре

In [2]:
import cv2
import numpy as np
import torch
import torchvision.transforms as transforms
from torchvision.models.detection import fasterrcnn_resnet50_fpn

# Загрузка модели
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = fasterrcnn_resnet50_fpn(pretrained=True).to(device)
model.eval()

# Загрузка видеопотока
cap = cv2.VideoCapture(0)  # 0 означает использование первой камеры

# Определение класса "человек"
PERSON_CLASS_ID = 1

# Преобразования для входного изображения
transform = transforms.Compose([
    transforms.ToTensor()
])

# Координаты интересующей области (x1, y1, x2, y2)
area_x1, area_y1, area_x2, area_y2 = 0, 0, 100, 200

while True:
    ret, img = cap.read()

    # Обрезка кадра до заданной области
    roi = img[area_y1:area_y2, area_x1:area_x2]

    # Преобразование кадра
    image = transform(roi)
    image = image.unsqueeze(0)  # Добавляем batch dimension

    # Применение модели для обнаружения объектов
    with torch.no_grad():
        predictions = model(image)
    
    # Анализ результатов
    labels = predictions[0]['labels'].numpy()
    boxes = predictions[0]['boxes'].detach().numpy()
    scores = predictions[0]['scores'].detach().numpy()

    # Проверка наличия человека
    for label, box, score in zip(labels, boxes, scores):
        if label == PERSON_CLASS_ID and score > 0.5:
            x1, y1, x2, y2 = box.astype(int)
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(img, f'Person: {score:.2f}', (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    
    # Отображение кадра
    cv2.imshow('Frame', img)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Освобождение ресурсов
cap.release()
cv2.destroyAllWindows()


NameError: name 'FasterRCNN_ResNet50_FPN_Weights' is not defined

# Импортируемые библиотеки и инициализация базы данных

In [1]:
import cv2
import os
import pickle
import mediapipe as mp
import face_recognition
import numpy as np
import firebase_admin
from firebase_admin import credentials
from firebase_admin import db
from firebase_admin import storage
from datetime import datetime

cred = credentials.Certificate("serviceAccountKey.json")
firebase_admin.initialize_app(cred, {'databaseURL': "https://facedetectiondb-6bd7e-default-rtdb.firebaseio.com/",
                                     'storageBucket': 'facedetectiondb-6bd7e.appspot.com'})

# Функция по нахождению угла между тремя точками на теле

In [2]:
def calculate_angle(a,b,c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0/np.pi)

    if angle > 180.0:
        angle = 360 - angle

    return angle

# Основная часть кода

In [ ]:
# Инициализация моделей для определения костей
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# Переменные для подсчета присядов
stage = "up"
sit_counter = 0

# Инициализация сущностей базы данных
ref = db.reference('Employees')

# Добавление лиц в базу данных
data = {
    "963852":
        {
            'name': 'Elon Musk',
            'amount_of_walk_aways': 0,
            'last_time_attendance': '2024-07-11 00:07:23',
            'counter': 0
        },
    "185249":
        {
            'name': 'Eliot Alderson',
            'amount_of_walk_aways': 0,
            'last_time_attendance': '2024-07-11 00:07:23',
            'counter': 0
        },
}

for key, value in data.items():
    ref.child(key).set(value)

# Импорт лиц для базы данных

folderPath = 'Images'
PathList = os.listdir(folderPath)
imgList = []
employeeID = []
for path in PathList:
    imgList.append(cv2.imread(os.path.join(folderPath, path)))
    employeeID.append(os.path.splitext(path)[0])

    fileName = f'{folderPath}/{path}'
    bucket = storage.bucket()
    blob = bucket.blob(fileName)
    blob.upload_from_filename(fileName)


def findEncodings(imagesList):
    encodeList = []
    for img in imagesList:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(img)[0]
        encodeList.append(encode)

    return encodeList

print('Encoding Started')
encodeListKnown = findEncodings(imgList)
encodeListKnownIds = [encodeListKnown, employeeID]
print('Encoding Complete')

file = open("EncodeFile.p", 'wb')
pickle.dump(encodeListKnownIds, file)
file.close()
print('File Saved')

bucket = storage.bucket()

# Инициализация видео потока
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

imgBackground = cv2.imread('Resources/background.png')

folderModePath = 'Resources/Modes'
modePathList = os.listdir(folderModePath)
imgModeList = []
for path in modePathList:
    imgModeList.append(cv2.imread(os.path.join(folderModePath, path)))

# Загрузка файла кодировки
print('Loading encode file')
file = open('EncodeFile.p', 'rb')
encodeListKnownIds = pickle.load(file)
file.close()
encodeListKnown, employeeID = encodeListKnownIds

print('Encode file loaded')


modeType = 0
counter = 0
id = -1
imageEmployee = []

# Основной цикл
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while True:
        # Переменные для захвата видео
        success, img = cap.read()

        # Конвертирование видеоряда
        imgS = cv2.resize(img, (0, 0), None, 0.25, 0.25)
        imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)
    
        faceCurFrame = face_recognition.face_locations(imgS)
        encodeCurFrame = face_recognition.face_encodings(imgS, faceCurFrame)
    
        imgBackground[162:162 + 480, 55:55 + 640] = img
        imgBackground[44:44 + 633, 808:808 + 414] = imgModeList[modeType]
    
        for encodeFace, faceLoc in zip(encodeCurFrame, faceCurFrame):
            matches = face_recognition.compare_faces(encodeListKnown, encodeFace)
            faceDis = face_recognition.face_distance(encodeListKnown, encodeFace)
    
            matchIndex = np.argmin(faceDis)
            if matches[matchIndex]:
                print(employeeID[matchIndex])
                id = employeeID[matchIndex]
                if counter == 0:
                    counter = 1
                    modeType = 1

        # Блок для опеределения костей и нахождения угла между ними
        imgBackground = cv2.cvtColor(imgBackground, cv2.COLOR_BGR2RGB)
        imgBackground.flags.writeable = False
        results = pose.process(imgBackground)
        imgBackground.flags.writeable = True
        imgBackground = cv2.cvtColor(imgBackground, cv2.COLOR_RGB2BGR)
        employeeInfo = db.reference(f'Employees/{id}').get()
        try:         
            landmarks = results.pose_landmarks.landmark
            
            # Получение координат
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            
            left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            
            right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
            right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            
            # Вычисление углов
            angle_left_hip = calculate_angle(left_shoulder, left_hip, left_knee)
            angle_right_hip = calculate_angle(right_shoulder, right_hip, right_knee)
            angle_left_knee = calculate_angle(left_ankle, left_knee, left_hip)
            angle_right_knee = calculate_angle(right_ankle, right_knee, right_hip)
            print(angle_left_hip)
            print(angle_right_hip)
            print(angle_left_knee)
            print(angle_right_knee)

            cv2.putText(imgBackground, str(int(angle_left_hip)), tuple(np.multiply(left_hip, [640, 480]).astype(int)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2, cv2.LINE_AA)
            cv2.putText(imgBackground, str(int(angle_right_hip)), tuple(np.multiply(right_hip, [640, 480]).astype(int)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2, cv2.LINE_AA)
            cv2.putText(imgBackground, str(int(angle_left_knee)), tuple(np.multiply(left_knee, [640, 480]).astype(int)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2, cv2.LINE_AA)
            cv2.putText(imgBackground, str(int(angle_right_knee)), tuple(np.multiply(right_knee, [640, 480]).astype(int)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2, cv2.LINE_AA)
            
            # Логика подсчета присядов
            if (angle_left_hip < 110 or angle_right_hip < 110 or angle_left_knee < 110 or angle_right_knee < 110) and stage == "up":
                stage = "down"
                ref = db.reference(f'Employees/{id}')
                employeeInfo['amount_of_walk_aways'] += 1
                ref.child('amount_of_walk_aways').set(employeeInfo['amount_of_walk_aways'])
                sit_counter += 1
            if angle_left_hip > 160 and angle_right_hip > 160 and angle_left_knee > 160 and angle_right_knee > 160 and stage =="down":
                stage = "up"              
            
        except:
            pass

        cv2.rectangle(imgBackground, (0,0), (215, 72), (50, 205, 50), -1)

        cv2.putText(imgBackground, 'SITS COUNT', (15,15), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(imgBackground, str(sit_counter), (10,55), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 2, cv2.LINE_AA)

        cv2.putText(imgBackground, 'STAGE', (90,15), cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(imgBackground, stage, (90,55), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 2, cv2.LINE_AA)

        # Вывод вычислений
        mp_drawing.draw_landmarks(imgBackground, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
    
        if counter != 0:
    
            if counter == 1:
                # Получение данных
                employeeInfo = db.reference(f'Employees/{id}').get()
                print(employeeInfo)
                # Получение изображения
                blob = bucket.get_blob(f'Images/{id}.png')
                array = np.frombuffer(blob.download_as_string(), dtype=np.uint8)
                imageEmployee = cv2.imdecode(array, cv2.COLOR_BGRA2BGR)
                # Обновление базы данных
                datetimeObject = datetime.strptime(employeeInfo['last_time_attendance'], "%Y-%m-%d %H:%M:%S")
                timeElapsed = (datetime.now() - datetimeObject).total_seconds()
                print(timeElapsed)
    
                ref = db.reference(f'Employees/{id}')
                employeeInfo['amount_of_walk_aways'] += 1
                ref.child('amount_of_walk_aways').set(employeeInfo['amount_of_walk_aways'])
                ref.child('last_time_attendance').set(datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    
                cv2.putText(imgBackground, str(employeeInfo['amount_of_walk_aways']), (400,300), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255,255,255), 2)

            # Изменения HUD элементов
            if 10 < counter <= 20:
                modeType = 2
    
            imgBackground[44:44 + 633, 808:808 + 414] = imgModeList[modeType]
    
            if counter <= 10:
    
                cv2.putText(imgBackground, str(employeeInfo['amount_of_walk_aways']), (861, 125),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 1)
                cv2.putText(imgBackground, str(id), (1006, 493),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
    
                (w, h), _ = cv2.getTextSize(employeeInfo['name'], cv2.FONT_HERSHEY_SIMPLEX, 1, 1)
                offset = (414 - w) // 2
                cv2.putText(imgBackground, str(employeeInfo['name']), (808 + offset, 445),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 1)
    
                imgBackground[175:175 + 216, 909:909 + 216] = imageEmployee
    
            counter += 1
    
            if counter >= 20:
                counter = 0
                modeType = 0
                employeeInfo = []
                imageEmployee = []
                imgBackground[44:44 + 633, 808:808 + 414] = imgModeList[modeType]

        # Отображение кадра
        cv2.imshow('Face Attendance', imgBackground)
        cv2.waitKey(1)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Освобождение ресурсов
cap.release()
cv2.destroyAllWindows()